# **Python Azure 02**
---

💡 File Storage에 파일을 올리고, 확인한 후 Local Storage로 다운로드 받는 과정을 실습해본다. 그리고 스냅샷을 생성하고 생성했던 스냅샷, 파일 및 공유 폴더를 삭제해본다.


- 실습을 위해 필요한 패키지를 설치한다.

In [61]:
!pip install azure-storage-file

- Azure Storage Account의 이름(`account_name`)과 액세스 키(`account_key`)를 불러와 File Share와 연결을 시켜준다.

In [31]:
from azure.storage.file import FileService
file_service = FileService(account_name='mylabuserstorage', account_key='*******************************************')    # 액세스 키 ✅

- File Storage에 새로운 폴더(`myshare`)를 하나 만든다.

In [32]:
file_service.create_share('myshare')

True

- File Storage에 새로운 디렉터리를 생성한다. (`myshare` 폴더에 `sampledir` 폴더를 만든다.)

In [33]:
file_service.create_directory('myshare', 'sampledir')

True

- Local Storage의 이미지 파일(`iu.jfif`)을 `myfile.jfif`로 이름을 바꾸어 File Storage의 `sampledir` 폴더에 업로드한다.

In [41]:
from azure.storage.file import ContentSettings
file_service.create_file_from_path(
    'myshare',
    'sampledir',
    'myfile.jfif',
    'iu.jfif'
)

- File Storage의 `myshare` 디렉터리 내부 파일들의 목록을 출력한다.

In [42]:
# Directory 내용의 확인 
generator = file_service.list_directories_and_files('myshare')
for file_or_dir in generator:
  print(file_or_dir.name)

sampledir


- File Storage의 파일을 Local Storage로 다운로드한다.
    - 다운로드를 하고 파일의 이름을 `IU_DOWNLOAD.jfif`로 바꾼다.

In [43]:
# 파일을 가져오는 코드
file_service.get_file_to_path('myshare', 'sampledir', 'myfile.jfif', 'IU_DOWNLOAD.jfif')

- 스냅샷을 만들어보고 확인해본다.

In [44]:
# Snapshot
metadata = { 'foo' : 'bar' }
snapshot = file_service.snapshot_share('myshare', metadata=metadata)

In [56]:
# 확인
share = list(file_service.list_shares(include_snapshots=True))

directories_and_files = list(file_service.list_directories_and_files('myshare', snapshot='2023-05-14T12:09:25.0000000Z'))   # 지정된 스냅샷으로 돌아갈 수 있다.

for file_or_dir in directories_and_files:
    print(file_or_dir.name)

sampledir


- 스냅샷을 삭제해본다.

In [57]:
# 스냅샷 삭제
file_service.delete_share('myshare', snapshot='2023-05-14T12:09:25.0000000Z')    # 특정 스냅샷만 삭제

True

- 지금까지 생성했던 **파일**을 삭제해본다. (`myshare/sampledir/myfile.jfif`)

In [46]:
file_service.delete_file('myshare', 'sampledir', 'myfile.jfif')

- 생성했던 **공유 폴더** 자체를 삭제한다.
    - 공유 폴더에 스냅샷이 들어있으면 삭제가 되지 않으니, 반드시 공유 폴더 안에 있는 스냅샷을 모두 지워준 후 이 작업을 수행한다.

In [60]:
file_service.delete_share('myshare')     

True